In [3]:
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel, BertTokenizer

In [4]:
tokenizer = BertTokenizer.from_pretrained('uer/gpt2-distil-chinese-cluecorpussmall')
model = GPT2LMHeadModel.from_pretrained('uer/gpt2-distil-chinese-cluecorpussmall')
print(model)

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

c:\Users\CNZHLIU14\Anaconda3\envs\nlp\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\CNZHLIU14\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/217 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/577 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/421M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(21128, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [5]:
inputs = tokenizer("得了头疼病怎么办")['input_ids']
print(inputs)
print(tokenizer.decode(inputs))

[101, 2533, 749, 1928, 4563, 4567, 2582, 720, 1215, 102]
[CLS] 得 了 头 疼 病 怎 么 办 [SEP]


## Train your model

In [1]:
from glob import glob
from transformers import DataCollatorForLanguageModeling
from datasets import Dataset
import os

In [2]:
os.environ

environ{'ALLUSERSPROFILE': 'C:\\ProgramData',
        'APPDATA': 'C:\\Users\\CNZHLIU14\\AppData\\Roaming',
        'CHROME_CRASHPAD_PIPE_NAME': '\\\\.\\pipe\\LOCAL\\crashpad_32132_IOGNYMQILSBPKEHU',
        'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files',
        'COMMONPROGRAMFILES(X86)': 'C:\\Program Files (x86)\\Common Files',
        'COMMONPROGRAMW6432': 'C:\\Program Files\\Common Files',
        'COMPUTERNAME': 'CN-L-7352565',
        'COMSPEC': 'C:\\WINDOWS\\system32\\cmd.exe',
        'CONDA_DEFAULT_ENV': 'nlp',
        'CONDA_EXE': 'C:\\Users\\CNZHLIU14\\Anaconda3\\Scripts\\conda.exe',
        'CONDA_EXES': '"C:\\Users\\CNZHLIU14\\Anaconda3\\condabin\\..\\Scripts\\conda.exe"  ',
        'CONDA_PREFIX': 'C:\\Users\\CNZHLIU14\\Anaconda3\\envs\\nlp',
        'CONDA_PROMPT_MODIFIER': '(nlp) ',
        'CONDA_PYTHON_EXE': 'C:\\Users\\CNZHLIU14\\Anaconda3\\python.exe',
        'CONDA_ROOT': 'C:\\Users\\CNZHLIU14\\Anaconda3',
        'CONDA_SHLVL': '1',
        'CUDA_PATH': '

In [ ]:
# os.environ['CUDA_VISIBLE_DEVICES'] = "3"

In [4]:
def read(path):
    result = []
    with open(path) as f:
        lines = [eval(s.strip()) for s in f.readlines()]
    for sentence,label in lines:
        result.append(sentence + label)
    return result

In [5]:
def tokenize(element):
    # 对句子进行分词，短的补0 长的阶段，长度统一为context_length
    outputs = tokenizer(element['inputs'], 
                        padding='max_length', 
                        truncation=True,
                        max_length='context_length', 
                        return_overflowing_tokens=True,
                        return_length=True)
    input_batch = []
    count = 0
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        input_batch.append(input_ids)
    return {
        "inputs": input_batch
    }
    

In [ ]:
# 读取数据
# 拼接问题描述和答案, 形成一个句子, 然后多个问答对就形成了多个问答句子

train_texts = read("train_data_QA")

In [ ]:
raw_datasets = Dataset.from_dict({'inputs': train_texts})
context_length = 200

In [ ]:
# 加载分词器
tokenizer = BertTokenizer.from_pretrained("uer/gpt2-distil-chinese-cluecorpussmall")

# 使用分词器对训练数据进行分词
tokenized_datasets = raw_datasets.map(tokenize, batched=True)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# gpt2中文版, 数据集: cluecorpussmall, distil蒸馏
model = GPT2LMHeadModel.from_pretrained("uer/gpt2-distil-chinese-cluecorpussmall")

# 训练的标配参数, 一般不用修改
args = TrainingArgments()
